In [1]:
import requests # to get the html markdown from the url
from bs4 import BeautifulSoup # to make the html readable
import pandas as pd # to save table in a csv
from datetime import datetime # to append current time into file name


# List of all IG urls

In [2]:
url = 'https://sinca.mma.gob.cl/'
html_data = requests.get(url)
soup = BeautifulSoup(html_data.text, 'html.parser')

# list of all regions link. Each region page has their stations link. From station link we get the data.
regiones = ['https://sinca.mma.gob.cl/'+x['href'] for x in soup.findAll('a')][15:-4]


In [29]:
regiones

['https://sinca.mma.gob.cl//index.php/region/index/id/XV',
 'https://sinca.mma.gob.cl//index.php/region/index/id/I',
 'https://sinca.mma.gob.cl//index.php/region/index/id/II',
 'https://sinca.mma.gob.cl//index.php/region/index/id/III',
 'https://sinca.mma.gob.cl//index.php/region/index/id/IV',
 'https://sinca.mma.gob.cl//index.php/region/index/id/V',
 'https://sinca.mma.gob.cl//index.php/region/index/id/M',
 'https://sinca.mma.gob.cl//index.php/region/index/id/VI',
 'https://sinca.mma.gob.cl//index.php/region/index/id/VII',
 'https://sinca.mma.gob.cl//index.php/region/index/id/VIII',
 'https://sinca.mma.gob.cl//index.php/region/index/id/IX',
 'https://sinca.mma.gob.cl//index.php/region/index/id/XIV',
 'https://sinca.mma.gob.cl//index.php/region/index/id/X',
 'https://sinca.mma.gob.cl//index.php/region/index/id/XI',
 'https://sinca.mma.gob.cl//index.php/region/index/id/XII']

In [4]:
# running time: 32 to 42 seconds
# get list of urls. all station links

url_stations = []
for page in regiones:
    html_data = requests.get(page)
    soup = BeautifulSoup(html_data.text, 'html.parser')
    url_stations = url_stations + [x.findAll('a')[0]['href'] for x in soup.find_all('tbody')[0].find_all('tr')]

In [13]:
url_stations

['/index.php/estacion/index/id/232',
 '/index.php/estacion/index/id/157',
 '/index.php/estacion/index/id/259',
 '/index.php/estacion/index/id/154',
 '/index.php/estacion/index/id/70',
 '/index.php/estacion/index/id/33',
 '/index.php/estacion/index/id/46',
 '/index.php/estacion/index/id/234',
 '/index.php/estacion/index/id/207',
 '/index.php/estacion/index/id/222',
 '/index.php/estacion/index/id/275',
 '/index.php/estacion/index/id/124',
 '/index.php/estacion/index/id/282',
 '/index.php/estacion/index/id/134',
 '/index.php/estacion/index/id/12',
 '/index.php/estacion/index/id/108',
 '/index.php/estacion/index/id/69',
 '/index.php/estacion/index/id/3',
 '/index.php/estacion/index/id/68',
 '/index.php/estacion/index/id/2',
 '/index.php/estacion/index/id/1',
 '/index.php/estacion/index/id/279',
 '/index.php/estacion/index/id/4',
 '/index.php/estacion/index/id/66',
 '/index.php/estacion/index/id/255',
 '/index.php/estacion/index/id/71',
 '/index.php/estacion/index/id/72',
 '/index.php/estac

In [8]:
# running time 3m 30s to 4m 20s

rows=[]

for region in url_stations:
    
    url = 'https://sinca.mma.gob.cl'+region
    html_data = requests.get(url)
    soup = BeautifulSoup(html_data.text, 'html.parser')
    
    estacion = soup.select('h1')[1].text.split(' ')[1:]
    estacion = ' '.join(estacion)

    GI_cols =   [ x.get_text().strip() for x in soup.find_all('th',{'class':'right'})]
    GI_values = [ x.get_text().strip() for x in soup.find_all('td',{'class':'left'}) ]

    row={}
    for idx in range(len(GI_values)):
        row.update({GI_cols[idx] : GI_values[idx]})
    row.update({'Estacion_id':int(region.split('/')[-1])})
    row.update({'Estacion':estacion})
    row.update({'URL':url})
    
    rows.append(row)

data = pd.DataFrame(rows)

In [30]:
url

'https://sinca.mma.gob.cl/index.php/estacion/index/id/191'

In [27]:
data['URL'].apply(lambda x: x.split('/')[-1])

0      232
1      157
2      259
3      154
4       70
      ... 
208    281
209    166
210    238
211    264
212    191
Name: URL, Length: 213, dtype: object

In [28]:
data

,Propietario,Operador,Región,Provincia,Comuna,Coordenadas UTM,Huso horario,Recepción de datos,Inicio de operación reportada,Estacion_id,Estacion,URL,Fin de operación reportada,code
0,Ministerio del Medio Ambiente,Algoritmos y Mediciones Ambientales SpA,de Arica y Parinacota,Arica,Arica,363747 E 7955831 N,19,en línea,2013-11-27,232,Arica,https://sinca.mma.gob.cl/index.php/estacion/in...,NaN,232
1,Ministerio del Medio Ambiente,Algoritmos y Mediciones Ambientales SpA,de Tarapacá,Iquique,Alto Hospicio,385118 E 7755989 N,19,en línea,2015-12-17,157,Alto Hospicio,https://sinca.mma.gob.cl/index.php/estacion/in...,NaN,157
2,Ministerio del Medio Ambiente,Algoritmos y Mediciones Ambientales SpA,de Antofagasta,Antofagasta,Antofagasta,358874 E 7387875 N,19,en línea,2013-06-01,259,Antofagasta,https://sinca.mma.gob.cl/index.php/estacion/in...,NaN,259
3,Convenio AIA-CONAMA-SEREMI de Salud,Serpram S.A,de Antofagasta,Antofagasta,Antofagasta,357923 E 7383182 N,19,carga manual,2005-08-23,154,Oncológico,https://sinca.mma.gob.cl/index.php/estacion/in...,NaN,154
4,Convenio AIA-CONAMA-SEREMI de Salud,Serpram S.A,de Antofagasta,Antofagasta,Antofagasta,356498 E 7381134 N,19,carga manual,2007-09-01,70,Playa Blanca,https://sinca.mma.gob.cl/index.php/estacion/in...,NaN,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Sub Secretaría del Medio Ambiente,Algoritmos y Mediciones Ambientales SpA,de los Lagos,Llanquihue,Puerto Varas,669783 E 5422635 N,18,en línea,2020-04-27,281,Puerto Varas,https://sinca.mma.gob.cl/index.php/estacion/in...,NaN,281
209,Ministerio del Medio Ambiente,Ministerio del Medio Ambiente,Aysén del General Carlos Ibáñez del Campo,Aysén,Aysén,680977 E 4969547 N,18,en línea,2017-10-20,166,Vialidad,https://sinca.mma.gob.cl/index.php/estacion/in...,NaN,166
210,Ministerio del Medio Ambiente,Algoritmos y Mediciones Ambientales SpA,Aysén del General Carlos Ibáñez del Campo,Coihaique,Coyhaique,729281 E 4948421 N,18,en línea,2007-03-01,238,Coyhaique,https://sinca.mma.gob.cl/index.php/estacion/in...,NaN,238
211,Ministerio del Medio Ambiente,Algoritmos y Mediciones Ambientales SpA,Aysén del General Carlos Ibáñez del Campo,Coihaique,Coyhaique,730152 E 4948488 N,18,en línea,2013-09-01,264,Coyhaique II,https://sinca.mma.gob.cl/index.php/estacion/in...,NaN,264


In [ ]:
# Clean data

data = pd.DataFrame(rows)

data['Región'] = data['Región'].apply(lambda x: x.replace('de ',''))

data['Huso horario'] = data['Huso horario'].apply(lambda x: int(x))


In [ ]:
# SAVE TO EXCEL

date = datetime.now().strftime("%d-%m-%Y %H%M%S") # get local time as string

filePath = "../SINCA MMA Gob - Historical Data/Información general " + date + " - Webscrape.csv"

data.sort_values('Estacion', inplace=True)

data.to_csv(filePath, index=True) # save to file path

data